In [3]:
%load_ext autoreload
%autoreload 2
%load_ext watermark
%watermark -v -n -m -p numpy,scipy,sklearn,pandas

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark
Thu Jan 24 2019 

CPython 3.6.6
IPython 6.5.0

numpy 1.15.1
scipy 1.1.0
sklearn 0.19.1
pandas 0.23.4

compiler   : GCC 4.8.2 20140120 (Red Hat 4.8.2-15)
system     : Linux
release    : 4.20.3-arch1-1-ARCH
machine    : x86_64
processor  : 
CPU cores  : 16
interpreter: 64bit


In [23]:
%matplotlib inline
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
import nolds
import data
import mne

from data.data_files import CHANNEL_NAMES, DataKind, files_builder

PROJ_ROOT = os.getenv('THESIS_ROOT')
DATA_ROOT = os.path.abspath(os.path.join(PROJ_ROOT, 'data'))
PROCESSED_ROOT = os.path.abspath(os.path.join(DATA_ROOT, 'processed'))
RAW_ROOT = os.path.abspath(os.path.join(DATA_ROOT, 'raw'))
LABELED_ROOT = os.path.abspath(os.path.join(DATA_ROOT, 'labeled'))
DURATIONS_ROOT = os.path.abspath(os.path.join(DATA_ROOT, 'durations'))
REC_ROOT = os.path.abspath(os.path.join(DATA_ROOT, 'recplots'))
DIRECT_ROOT = os.path.abspath(os.path.join(DATA_ROOT, 'direct'))
GAF_ROOT = os.path.abspath(os.path.join(DATA_ROOT, 'gaf'))
print(PROJ_ROOT)
print(DATA_ROOT)
print(PROCESSED_ROOT)
import sys
sys.path.append(os.path.join(PROJ_ROOT, 'src'))
CHANNEL_NAMES = ['FP1', 'FP2', 'F3', 'F4', 'C3', 'C4', 'P3', 'P4', 'O1', 'O2',
                 'F7', 'F8', 'T3', 'T4', 'T5', 'T6', 'Fz', 'Cz', 'Pz']
META_COLUMN_NAMES = ['freq', 'RESP_4W', 'RESP_FIN', 'REMISE_FIN', 'AGE', 'SEX', 'M_1',
       'M_4', 'M_F', 'délka léčby', 'lék 1', 'lék 2', 'lék 3', 'lék 4']
META_FILE_NAME = 'DEP-POOL_Final_144.xlsx'
meta_df = pd.read_excel(os.path.join(RAW_ROOT, META_FILE_NAME), index_col='ID', names=META_COLUMN_NAMES)

raw_fif = mne.io.read_raw_fif(os.path.join(PROCESSED_ROOT, '50a.fif'))
t = pd.DataFrame(raw_fif.get_data())
data = pd.DataFrame(np.transpose(t.values), columns=CHANNEL_NAMES).values

/home/kovar/thesis_project/
/home/kovar/thesis_project/data
/home/kovar/thesis_project/data/processed
Opening raw data file /home/kovar/thesis_project/data/processed/50a.fif...
This filename (/home/kovar/thesis_project/data/processed/50a.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, raw.fif.gz, raw_sss.fif.gz or raw_tsss.fif.gz
Isotrak not found
    Range : 0 ... 16930 =      0.000 ...    67.720 secs
Ready.


<ipython-input-23-708761e1ae21>:34: RuntimeWarning: This filename (/home/kovar/thesis_project/data/processed/50a.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, raw.fif.gz, raw_sss.fif.gz or raw_tsss.fif.gz
  raw_fif = mne.io.read_raw_fif(os.path.join(PROCESSED_ROOT, '50a.fif'))


In [24]:
# Welford's algorithm for computing running mean and variance
def update(existingAggregate, newValues):
    (count, mean, M2) = existingAggregate
    for newValue in newValues: 
        count += 1
        delta = newValue - mean
        mean += delta / count
        delta2 = newValue - mean
        M2 += delta * delta2
        existingAggregate = (count, mean, M2)

    return (count, mean, M2)

def finalize(existingAggregate):
    (count, mean, M2) = existingAggregate
    (mean, variance) = (mean, M2/count) 
    if count < 2:
        return float('nan')
    else:
        return (mean, np.sqrt(variance))

In [25]:
# Algos
from scipy.spatial.distance import pdist, squareform
import math

def rec_plot(data):
    if len(data.shape) > 1:
        return squareform(pdist(data)).astype('float64')
    else:
        return squareform(pdist(data[:,None])).astype('float64')

def tabulate(x, y, f):
    return np.vectorize(f)(*np.meshgrid(x, y, sparse=True))

def cos_sum(x, y):
    return math.cos(x+y)

def gaf(serie):
    # Min-Max scaling
    min_ = np.amin(serie)
    max_ = np.amax(serie)
    scaled_serie = (2*serie - max_ - min_)/(max_ - min_)

    # Floating point inaccuracy!
    scaled_serie = np.where(scaled_serie >= 1., 1., scaled_serie)
    scaled_serie = np.where(scaled_serie <= -1., -1., scaled_serie)

    # Polar encoding
    phi = np.arccos(scaled_serie)

    # GAF Computation (every term of the matrix)
    gaf = tabulate(phi, phi, cos_sum)

    return gaf

# Compute recurrence plot / GAF (multichannel distance)

In [18]:
count, mean, M2 = 0, 0, 0

def compute_vec(file, f, path, ww=256, maxl=np.infty):
    global count, mean, M2
    minl = ww
    start = 0
    chunk_num = 0
    # for fn in os.listdir(path):
    #     no_ext, _ = os.path.splitext(file.name)
    #     if fn.startswith(no_ext):
    #         print('Returning None, file ', no_ext, 'at teast partially processed')
    #         return None
    while start+ww <= min(maxl, len(file.df['FP1'].values)):
        # Add third dimension to make Keras happy
        r = np.zeros((ww, ww, 1))
        data = file.df.values
        # Here we may select only a subset of channels, let's try all for now
        r[:,:,0] = f(data[start:start+ww, :])
        if 2*len(data) < minl + ww or r.shape[0]*r.shape[1] != ww*ww:
            print('Returning None, for file ', file.name, ', time series too short: ', len(data))
            print('Or returned wrong shape: ', r.shape, start+ww)
            return None
        count, mean, M2 = update((count, mean, M2), r.reshape(-1))
        np.save(
            os.path.join(path, ''.join((str(file.id), file.trial, '-', str(chunk_num), '.npy'))), r, fix_imports=False)
        start += ww
        chunk_num += 1
    return r

In [139]:
import logging
mne.set_log_level(logging.ERROR)
count, mean, M2 = 0, 0, 0
for i, file in enumerate(files_builder(DataKind('processed'))):
    compute_vec(file, rec_plot, os.path.join(REC_ROOT, 'vectors'))
    print('Processed: ', i)

Processed:  0
Processed:  1
Processed:  2
Processed:  3
Processed:  4
Processed:  5
Processed:  6
Processed:  7
Processed:  8
Processed:  9
Processed:  10
Processed:  11
Processed:  12
Processed:  13
Processed:  14
Processed:  15
Processed:  16
Processed:  17
Processed:  18
Processed:  19
Processed:  20
Processed:  21
Processed:  22
Processed:  23
Processed:  24
Processed:  25
Processed:  26
Processed:  27
Processed:  28
Processed:  29
Processed:  30
Processed:  31
Processed:  32
Processed:  33
Processed:  34
Processed:  35
Processed:  36
Processed:  37
Processed:  38
Processed:  39
Processed:  40
Processed:  41
Processed:  42
Processed:  43
Processed:  44
Processed:  45
Processed:  46
Processed:  47
Processed:  48
Processed:  49
Processed:  50
Processed:  51
Processed:  52
Processed:  53
Processed:  54
Processed:  55
Processed:  56
Processed:  57
Processed:  58
Processed:  59
Processed:  60
Processed:  61
Processed:  62
Processed:  63
Processed:  64
Processed:  65
Processed:  66
Proce

ValueError: not enough values to unpack (expected 3, got 2)

In [142]:
mean, st = finalize((count, mean, M2))
path = os.path.join(REC_ROOT, 'vectors')
for fn in os.listdir(path):
    filepath = os.path.join(path, fn)
    r = np.load(filepath)
    r = (r-mean) / std
    # assert (r >= -1).all() and (r <= 1).all()
    np.save(filepath, r, fix_imports=False)

# Compute recurrence plot / GAF (separate channels)

In [26]:
def compute_sep(file, f, path, ww=256, maxl=np.infty):
    minl = ww
    start = 0
    chunk_num = 0
    while start+ww <= min(maxl, len(file.df['FP1'].values)):
        for i, channel in enumerate(CHANNEL_NAMES):
            # file_found = False
            # for fn in os.listdir(os.path.join(path, channel)):
            #     no_ext, _ = os.path.splitext(file.name)
            #     if fn.startswith(no_ext):
            #         print('File ', fn, ' already processed, for channel ', channel, ', skipping...')
            #         file_found = True
            #         break
            # if file_found: continue
            data = file.df[channel].values
            r = f(data[start:start+ww])
            if 2*len(data) < minl + ww or r.shape[0]*r.shape[1] != ww*ww:
                print('Skipping, file ', file.name, ', time series too short: ', len(data))
                print('Or returned wrong shape: ', r.shape, start+ww)
                continue
            counts[i], means[i], M2s[i] = update((counts[i], means[i], M2s[i]), r.reshape(-1))
            np.save(
                os.path.join(path, channel, ''.join((str(file.id), file.trial, '-', str(chunk_num), '.npy'))),
                r, fix_imports=False)
        start += ww
        chunk_num += 1
    return r

In [ ]:
CHANNEL_NAMES = ['T6',]
counts, means, M2s = np.zeros(19), np.zeros(19), np.zeros(19)
final_means, final_stds = np.zeros(19), np.zeros(19)

for channel in CHANNEL_NAMES:
    if not os.path.exists(os.path.join(GAF_ROOT, 'sep_channels', channel)):
        os.makedirs(os.path.join(GAF_ROOT, 'sep_channels', channel))

import logging
mne.set_log_level(logging.ERROR)
for i, file in enumerate(files_builder(DataKind('processed'))):
    # compute_sep(file, rec_plot, os.path.join(REC_ROOT, 'sep_channels'))
    compute_sep(file, gaf, os.path.join(GAF_ROOT, 'sep_channels'))
    print('Processed: ', i)
    
print('Finalizing...')
for i in np.arange(len(CHANNEL_NAMES)):
    (final_means[i], final_stds[i]) = finalize((counts[i], means[i], M2s[i]))

Processed:  0
Processed:  1
Processed:  2
Processed:  3
Processed:  4
Processed:  5
Processed:  6
Processed:  7
Processed:  8
Processed:  9
Processed:  10
Processed:  11
Processed:  12
Processed:  13
Processed:  14
Processed:  15
Processed:  16
Processed:  17
Processed:  18
Processed:  19
Processed:  20
Processed:  21
Processed:  22
Processed:  23
Processed:  24
Processed:  25
Processed:  26
Processed:  27
Processed:  28
Processed:  29
Processed:  30
Processed:  31
Processed:  32
Processed:  33
Processed:  34
Processed:  35
Processed:  36
Processed:  37
Processed:  38
Processed:  39
Processed:  40
Processed:  41
Processed:  42
Processed:  43
Processed:  44
Processed:  45
Processed:  46
Processed:  47
Processed:  48
Processed:  49
Processed:  50
Processed:  51
Processed:  52
Processed:  53
Processed:  54
Processed:  55
Processed:  56
Processed:  57
Processed:  58
Processed:  59
Processed:  60
Processed:  61
Processed:  62
Processed:  63
Processed:  64
Processed:  65
Processed:  66
Proce

In [159]:
print('Saving...')
for i, channel in enumerate(CHANNEL_NAMES):
    path = os.path.join(REC_ROOT, 'sep_channels', channel)
    for fn in os.listdir(path):
        filepath = os.path.join(path,fn)
        r = np.load(filepath)
        r[:, :] = (r[:, :]-final_means[i])/final_stds[i]
        np.save(filepath, r, fix_imports=False)

Saving...


In [113]:
print('Computing means and variances...')
path = os.path.join(DATA_ROOT, 'recplots_1')
for fn in os.listdir(path):
    r = np.load(os.path.join(path,fn))
    rs = r.reshape((-1, 19))
    for i in np.arange(r.shape[2]):
        counts[i], means[i], M2s[i] = update((counts[i], means[i], M2s[i]), rs[:, i])
        
print('Finalizing...')
for i in np.arange(len(CHANNEL_NAMES)):
    (final_means[i], final_stds[i]) = finalize((counts[i], means[i], M2s[i]))
    
print('Saving...')
for fn in os.listdir(path):
    filepath = os.path.join(path,fn)
    r = np.load(filepath)
    for i in np.arange(r.shape[2]):
        r[:, :, i] = (r[:, :, i]-final_means[i])/final_stds[i]
    np.save(filepath, r, fix_imports=False)

Computing means and variances...


FileNotFoundError: [Errno 2] No such file or directory: '/home/kovar/thesis_project/data/recplots_1/46a-48.npy'

# Directly to normalized images

In [16]:
def direct(data):
    return data

In [26]:
count, mean, M2 = 0, 0, 0

def compute_dir(file, f, path, ww=256, maxl=np.infty):
    global count, mean, M2
    minl = ww
    start = 0
    chunk_num = 0
    # for fn in os.listdir(path):
    #     no_ext, _ = os.path.splitext(file.name)
    #     if fn.startswith(no_ext):
    #         print('Returning None, file ', no_ext, 'at teast partially processed')
    #         return None
    while start+ww <= min(maxl, len(file.df['FP1'].values)):
        # Add third dimension to make Keras happy
        r = np.zeros((ww, len(CHANNEL_NAMES), 1))
        data = file.df.values
        # Here we may select only a subset of channels, let's try all for now
        r[:,:,0] = f(data[start:start+ww, :])
        if 2*len(data) < minl + ww or r.shape[0]*r.shape[1] != ww*len(CHANNEL_NAMES):
            print('Returning None, for file ', file.name, ', time series too short: ', len(data))
            print('Or returned wrong shape: ', r.shape, start+ww)
            return None
        count, mean, M2 = update((count, mean, M2), r.reshape(-1))
        np.save(
            os.path.join(path, ''.join((str(file.id), file.trial, '-', str(chunk_num), '.npy'))), r, fix_imports=False)
        start += ww
        chunk_num += 1
    return r

In [28]:
count, mean, M2 = 0, 0, 0

import logging
mne.set_log_level(logging.ERROR)
for file in files_builder(DataKind('processed')):
    compute_dir(file, direct, os.path.join(DIRECT_ROOT))
    # compute(file, gaf, GAF_ROOT)

In [29]:
mean, std = finalize((count, mean, M2))
path = os.path.join(DIRECT_ROOT)
for fn in os.listdir(path):
    filepath = os.path.join(path, fn)
    r = np.load(filepath)
    r = (r-mean) / std
    # assert (r >= -1).all() and (r <= 1).all()
    np.save(filepath, r, fix_imports=False)

# Training

In [3]:
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
config.log_device_placement = True  # to log device placement (on which device the operation ran)
                                    # (nothing gets printed in Jupyter, only if you run it standalone)
sess = tf.Session(config=config)
set_session(sess)  # set this TensorFlow session as the default session for Keras

In [4]:
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense
from keras.utils import to_categorical
from keras.models import Sequential
from keras.wrappers.scikit_learn import KerasClassifier
from keras import optimizers
from keras import initializers

import numpy as np
from sklearn.model_selection import GridSearchCV, train_test_split

from data.data_files import CHANNEL_NAMES, DataKind, files_builder
# from measures import algorithms as algos

from keras import backend as K
K.set_floatx('float32')
K.floatx()

'float32'

In [5]:
# image_height, image_width = 256, 256
image_height, image_width = 256, 19
num_channels = 1
batch_size = 128
num_epochs= 35
dropout_rate = 0.5

In [6]:
from keras.utils import Sequence

ind = [False, False, False, False, False,
       False, False, False, False, False, 
       False, False, False, False, False, 
       True, False, False, False]

print(np.array(CHANNEL_NAMES)[ind])

class batch_generator(Sequence):

    def __init__(self, filenames, labels, batch_size):
        self.filenames, self.labels = filenames, labels
        self.n = len(self.filenames)
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self.filenames) / float(self.batch_size)))
        # return len(self.filenames) // self.batch_size

    def __getitem__(self, idx):
        batch_x = self.filenames[idx * self.batch_size:min(self.n, (idx + 1) * self.batch_size)]
        batch_y = self.labels[idx * self.batch_size:min(self.n, (idx + 1) * self.batch_size)]
        # assert len(batch_x) == batch_size, batch_x
        # assert len(batch_y) == batch_size, batch_y

        return np.array([
            np.load(file_name) \
            # np.expand_dims(np.load(file_name), axis=-1) \
            # np.expand_dims(np.mean(np.load(file_name)[:, :, ind], axis=-1), axis=-1) \
            for file_name in batch_x]), batch_y

['T6']


In [9]:
def define_model_1(dropout_rate=dropout_rate):
    model = Sequential()
    model.add(Conv2D(8, (3, 3), input_shape=(image_height, image_width, num_channels), data_format='channels_last'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(16, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(16, (3, 3)))
    model.add(Activation('relu'))
    # model.add(MaxPooling2D(pool_size=(2, 2)))
    
    # model.add(Conv2D(4, (3, 3)))
    # model.add(Activation('relu'))
    # model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())
    model.add(Dropout(dropout_rate))
    model.add(Dense(64))
    model.add(Activation('relu'))
    model.add(Dense(16))
    model.add(Activation('relu'))
    model.add(Dense(2))
    model.add(Activation('sigmoid'))

    model.compile(loss='binary_crossentropy',
                  optimizer='rmsprop',
                  metrics=['accuracy'])
    
    return model
define_model_1().summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 254, 17, 8)        80        
_________________________________________________________________
activation_4 (Activation)    (None, 254, 17, 8)        0         
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 127, 8, 8)         0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 125, 6, 16)        1168      
_________________________________________________________________
activation_5 (Activation)    (None, 125, 6, 16)        0         
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 62, 3, 16)         0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 60, 1, 16)         2320      
__________

In [10]:
# VGG-like model
def define_model_2():
    model = Sequential()
    model.add(Conv2D(8, (3, 3), activation='relu', input_shape=(image_height,image_width,num_channels)))
    model.add(Conv2D(16, (3, 3), activation='relu'))
    model.add(MaxPooling2D())
    model.add(Conv2D(32, (3, 3), activation='relu'))
    model.add(Conv2D(32, (3, 3), activation='relu'))
    model.add(MaxPooling2D())
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(2, activation='softmax'))
    model.compile(optimizer='Adam',
                  # optimizer='rmsprop',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

In [18]:
# Nikita's model
def define_model_3():
    model = Sequential()
    ki = initializers.RandomNormal(0, 0.1, 23)
    model.add(Conv2D(16, (3, 3), activation='relu', input_shape=(image_height,image_width,num_channels),
                     kernel_initializer=ki))
    model.add(Conv2D(16, (3, 3), activation='relu', kernel_initializer=ki))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.75))
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer=ki))
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer=ki))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.75))
    model.add(Flatten())
    model.add(Dense(128, activation='relu', kernel_initializer=ki))
    model.add(Dropout(0.5))
    model.add(Dense(2, activation='sigmoid'))
    # model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer=optimizers.Adam(lr=1e-4, beta_1=0.999, beta_2=0.999, epsilon=1e-8, decay=0.0, amsgrad=False),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

In [19]:
# From 'Classification of Recurrence Plots’ Distance Matrices with a Convolutional Neural Network for
# Activity Recognition' paper
def define_model_4():
    model = Sequential()
    ki = initializers.RandomNormal()
    model.add(Conv2D(16, (3, 3), activation='relu', input_shape=(image_height,image_width,num_channels),
                     kernel_initializer=ki))
    model.add(Conv2D(16, (3, 3), activation='relu', kernel_initializer=ki))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer=ki))
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer=ki))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(512, activation='relu', kernel_initializer=ki))
    model.add(Dropout(0.5))
    model.add(Dense(2, activation='softmax'))
    model.compile(optimizer=optimizers.Adam(lr=1e-3, beta_1=0.9, beta_2=0.999, epsilon=1e-8, decay=0.0, amsgrad=False),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

In [20]:
def remove_middle(filenames, labels):
    ta = zip(filenames, labels)
    ta = np.array([(t, l) for t, l in ta], dtype=[('fname', 'S100'), ('label', 'int8')])
    ta = ta[:][(ta['label'] == -1) | (ta['label'] == 1)]
    ta['label'][ta['label'] == -1] = 0
    return ta['fname'].astype(str, copy=False), ta['label']

In [14]:
fb = files_builder(DataKind('direct'), subfolder=())
seed = 123
fns = [fn[1] for fn in fb.file_names(include_path=True, subfolder=(), recursive=True)]
filenames, labels = remove_middle(fns, fb.get_labels(fns))
# labels = to_categorical(labels)
unique, counts = np.unique(labels, return_counts=True)
print('Overall distribution: ', dict(zip(unique, counts)))
training_filenames, validation_filenames, training_labels, validation_labels = \
    train_test_split(filenames, labels, test_size=0.2, random_state=seed)
assert len(training_filenames) == len(training_labels)
assert len(validation_filenames) == len(validation_labels)
unique, counts = np.unique(training_labels, return_counts=True)
print('Training distribution: ', dict(zip(unique, counts)))
unique, counts = np.unique(validation_labels, return_counts=True)
print('Testing distribution: ', dict(zip(unique, counts)))
training_labels = to_categorical(training_labels, dtype=training_labels.dtype)
validation_labels = to_categorical(validation_labels, dtype=validation_labels.dtype)

Overall distribution:  {0: 6801, 1: 6006}
Training distribution:  {0: 5427, 1: 4818}
Testing distribution:  {0: 1374, 1: 1188}


In [22]:
training_batch_generator = batch_generator(training_filenames, training_labels, batch_size)
validation_batch_generator = batch_generator(validation_filenames, validation_labels, batch_size)

model = define_model_4()
model.fit_generator(generator=training_batch_generator,
                                      # steps_per_epoch=(len(training_filenames) // batch_size),
                                      steps_per_epoch=len(training_batch_generator),
                                      epochs=num_epochs,
                                      verbose=1,
                                      validation_data=validation_batch_generator,
                                      # validation_steps=(len(validation_filenames) // batch_size),
                                      validation_steps=len(validation_batch_generator),
                                      use_multiprocessing=True,
                                      workers=8,
                                      max_queue_size=32)

Epoch 1/35
81/81 [==============================] - 5s 56ms/step - loss: 0.6896 - acc: 0.5363 - val_loss: 0.6825 - val_acc: 0.5589
Epoch 2/35
81/81 [==============================] - 3s 40ms/step - loss: 0.6796 - acc: 0.5538 - val_loss: 0.6840 - val_acc: 0.5550
Epoch 3/35
81/81 [==============================] - 3s 41ms/step - loss: 0.6752 - acc: 0.5714 - val_loss: 0.6776 - val_acc: 0.5621
Epoch 4/35
81/81 [==============================] - 3s 41ms/step - loss: 0.6645 - acc: 0.5862 - val_loss: 0.6583 - val_acc: 0.5867
Epoch 5/35
81/81 [==============================] - 3s 41ms/step - loss: 0.6522 - acc: 0.6004 - val_loss: 0.6551 - val_acc: 0.5999
Epoch 6/35
81/81 [==============================] - 3s 41ms/step - loss: 0.6380 - acc: 0.6142 - val_loss: 0.6362 - val_acc: 0.6226
Epoch 7/35
81/81 [==============================] - 3s 40ms/step - loss: 0.6228 - acc: 0.6309 - val_loss: 0.6279 - val_acc: 0.6315
Epoch 8/35
81/81 [==============================] - 3s 42ms/step - loss: 0.6113 - a

In [35]:
def get_model_memory_usage(batch_size, model):
    import numpy as np
    from keras import backend as K

    shapes_mem_count = 0
    for l in model.layers:
        single_layer_mem = 1
        for s in l.output_shape:
            if s is None:
                continue
            single_layer_mem *= s
        shapes_mem_count += single_layer_mem

    trainable_count = np.sum([K.count_params(p) for p in set(model.trainable_weights)])
    non_trainable_count = np.sum([K.count_params(p) for p in set(model.non_trainable_weights)])

    number_size = 4.0
    if K.floatx() == 'float16':
         number_size = 2.0
    if K.floatx() == 'float64':
         number_size = 8.0

    print('Total paramaters: {:,}'.format(trainable_count + non_trainable_count))
    total_memory = number_size*(batch_size*shapes_mem_count + trainable_count + non_trainable_count)
    gbytes = np.round(total_memory / (1024.0 ** 3), 3)
    return gbytes
get_model_memory_usage(128, define_model_1())

Total paramaters: 12,694.0


2.518